<a href="https://colab.research.google.com/github/KarelZe/thesis/blob/intermediate-results/notebooks/4.0b-mb-tabtransformer_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install catboost==1.1
!pip install gcsfs==2022.10.0
!pip install ipywidgets==8.0.2
!pip install numpy==1.23.4
!pip install pandas==1.5.1
!pip install scikit-learn==1.1.3
!pip install fastparquet
!pip install wandb
!pip install einops
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.7 MB 7.1 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 138 kB 18.7 MB/s 
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.11.0
    Uninstalling fsspec-2022.11.0:
      Successfully uninstalled fsspec-2022.11.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 134 kB 14.7 MB/s 
     |████████████████████████████████| 2.0 MB 73.8 MB/s 
     |████████████████████████████████| 1.6 MB 58.1 MB/s 
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.1
    Uninstalling widgetsnbextension-3.6.1:
      Successfully uninstalled widgetsnbextension-3.6.1
  Attempting uninstall: ipywidgets
    Found exis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.2 MB 13.5 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 31.2 MB 1.4 MB/s 
     |████████████████████████████████| 16.9 MB 57.3 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.4
    Uninstalling numpy-1.23.4:
      Successfully uninstalled numpy-1.23.4
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 14.8 MB/s 
     |████████████████████████████████| 2.0 MB 58.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 15.2 MB/s 
     |████████████████████████████████| 168 kB 70.2 MB/s 
     |████████████████████████████████| 182 kB 79.2 MB/s 
     |████████████████████████████████| 62 kB 1.8 MB/s 
     |████████████████████████████████| 168 kB 85.4 MB/s 
     |████████████████████████████████| 166 kB 80.6 MB/s 
     |████████████████████████████████| 166 kB 85.5 MB/s 
     |████████████████████████████████| 162 kB 86.0 MB/s 
     |████████████████████████████████| 162 kB 69.0 MB/s 
     |████████████████████████████████| 158 kB 82.5 MB/s 
     |████████████████████████████████| 157 kB 78.4 MB/s 
     |████████████████████████████████| 157 kB 84.

Run `pip install .` first to install all dependencies.

In [2]:
from catboost import CatBoostClassifier

import gcsfs
import google.auth
from google.colab import auth, output

import numpy as np
import pandas as pd

import os
from pathlib import Path

from sklearn.metrics import accuracy_score

from tqdm.notebook import tqdm
import wandb

from typing import List, Optional

In [3]:
# connect to google cloud storage
auth.authenticate_user()
credentials, _ = google.auth.default()
fs = gcsfs.GCSFileSystem(project="thesis", token=credentials)
fs_prefix = "gs://"

In [4]:
features_classical_size = [
    'TRADE_PRICE', 'bid_ask_size_ratio_ex', 'rel_bid_size_ex',
       'rel_ask_size_ex', 'TRADE_SIZE', 'bid_size_ex', 'ask_size_ex',
       'rel_ask_ex', 'rel_bid_ex', 'BEST_rel_bid', 'BEST_rel_ask',
       'bid_ask_ratio_ex', 'chg_ex_lead', 'chg_ex_lag', 'chg_all_lead',
       'chg_all_lag', 'ask_ex', 'bid_ex', 'BEST_ASK', 'BEST_BID',
       'price_all_lag', 'price_all_lead', 'price_ex_lag', 'price_ex_lead'
]

In [5]:
# see https://wandb.ai/fbv/thesis/runs/kwlaw02g/overview?workspace=user-karelze
# for refs

run = wandb.init(project="thesis",entity="fbv")

dataset = "fbv/thesis/classical_size_features_log_normalized:v0"
artifact = run.use_artifact(dataset)
data_dir = artifact.download()

model = "fbv/thesis/3lfsbuby_TabTransformer_default_trial_82.pth:v0"
artifact = run.use_artifact(model)
model_dir = artifact.download()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Downloading large artifact classical_size_features_log_normalized:v0, 2564.04MB. 3 files... 
wandb:   3 of 3 files downloaded.  
Done. 0:1:6.3
wandb:   1 of 1 files downloaded.  


In [6]:
X_test = pd.read_parquet(Path(data_dir, "test_set_20.parquet"), engine="fastparquet")

y_test = X_test["buy_sell"]
X_test = X_test[features_classical_size]


In [7]:
X_test.head()

,TRADE_PRICE,bid_ask_size_ratio_ex,rel_bid_size_ex,rel_ask_size_ex,TRADE_SIZE,bid_size_ex,ask_size_ex,rel_ask_ex,rel_bid_ex,BEST_rel_bid,...,chg_all_lead,chg_all_lag,ask_ex,bid_ex,BEST_ASK,BEST_BID,price_all_lag,price_all_lead,price_ex_lag,price_ex_lead
39342171,-0.886605,-0.999944,-0.999978,-0.999959,-0.920668,-0.397940,-0.568327,-0.450753,0.450753,-0.222609,...,0.979614,0.887864,-0.869878,-0.883935,-0.922989,-0.883936,-0.917824,-0.921640,-0.897066,-0.875702
39342172,-0.398830,-0.999980,-0.999660,-0.999773,-1.000000,-0.879588,-0.879588,-0.449826,0.449826,-0.221914,...,0.979609,0.887724,-0.387113,-0.403539,-0.637270,-0.403545,-0.511138,-0.615242,-0.380144,-0.521084
39342173,-0.060248,-0.999980,-0.999660,-0.999773,-1.000000,-0.879588,-0.879588,-0.450689,0.450689,-0.222561,...,0.979589,0.887492,-0.031559,-0.059053,-0.426840,-0.059063,-0.248786,-0.402171,-0.057769,-0.080426
39342174,-0.675788,-0.999980,-0.999966,-0.999977,-1.000000,-0.583443,-0.583443,-0.448436,0.448436,-0.220290,...,0.979647,0.887998,-0.673685,-0.709687,-0.811995,-0.709690,-0.802774,-0.914062,-1.000000,-1.000000
39342175,-0.727221,-0.999980,-0.999966,-0.999977,-1.000000,-0.583443,-0.583443,-0.450753,0.450753,-0.222609,...,0.979636,0.887758,-0.700498,-0.724674,-0.822743,-0.724677,-0.747425,-0.913006,-1.000000,-1.000000


## TabTransformer Baseline 🦾

In [8]:
"""
Implementation of a TabTransformer.
Based on paper:
https://arxiv.org/abs/2012.06678
Implementation adapted from: https://github.com/lucidrains/tab-transformer-pytorch
and https://github.com/kathrinse/TabSurvey/blob/main/models/tabtransformer.py
Also see: https://pytorch.org/docs/stable/_modules/torch/nn/modules/transformer.html
"""
from __future__ import annotations

from typing import Any, Callable

import torch
import torch.nn.functional as F
from einops import rearrange
from torch import einsum, nn


class Residual(nn.Module):
    """
    PyTorch implementation of residual connections.
    Args:
        nn (nn.Module): module
    """

    def __init__(self, fn: nn.Module):
        """
        Residual connection.
        Args:
            fn (nn.Module): network.
        """
        super().__init__()
        self.fn = fn

    def forward(self, x: torch.Tensor, **kwargs: Any) -> torch.Tensor:
        """
        Forward pass of residual connections.
        Args:
            x (torch.Tensor): input tensor.
        Returns:
            torch.Tensor: output tensor.
        """
        return self.fn(x, **kwargs) + x


class PreNorm(nn.Module):
    """
    PyTorch implementation of pre-normalization.
    Args:
        nn (nn.module): module.
    """

    def __init__(self, dim: int, fn: nn.Module):
        """
        Pre-normalization.
        Consists of layer for layer normalization followed by another network.
        Args:
            dim (int): Number of dimensions of normalized shape.
            fn (nn.Module): network.
        """
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn

    def forward(self, x: torch.Tensor, **kwargs: Any) -> torch.Tensor:
        """
        Forward pass of pre-normalization layers.
        Args:
            x (torch.Tensor): input tensor.
        Returns:
            torch.Tensor: output tensor.
        """
        return self.fn(self.norm(x), **kwargs)


class GEGLU(nn.Module):
    r"""
    Implementation of the GeGLU activation function.
    Given by:
    $\operatorname{GeGLU}(x, W, V, b, c)=\operatorname{GELU}(x W+b) \otimes(x V+c)$
    Proposed in https://arxiv.org/pdf/2002.05202v1.pdf.
    Args:
        nn (torch.Tensor): module
    """

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass of GeGlU activation.
        Args:
            x (torch.Tensor): input tensor.
        Returns:
            torch.Tensor: output tensor.
        """
        x, gates = x.chunk(2, dim=-1)
        return x * F.gelu(gates)


class FeedForward(nn.Module):
    """
    PyTorch implementation of feed forward network.
    Args:
        nn (nn.module): module.
    """

    def __init__(self, dim: int, mult: int = 4, dropout: float = 0.0):
        """
        Feed forward network.
        Network consists of input layer, GEGLU activation, dropout layer,
        and output layer.
        Args:
            dim (int): dimension of input and output layer.
            mult (int, optional): Scaling factor for output dimension of input layer or
            input dimension of output layer. Defaults to 4.
            dropout (float, optional): Degree of dropout. Defaults to 0.0.
        """
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, dim * mult * 2),
            GEGLU(),
            nn.Dropout(dropout),
            nn.Linear(dim * mult, dim),
        )

    def forward(self, x: torch.Tensor, **kwargs: Any) -> torch.Tensor:
        """
        Forward pass of feed forward network.
        Args:
            x (torch.Tensor): input tensor.
        Returns:
            torch.Tensor: output tensor.
        """
        return self.net(x)


class Attention(nn.Module):
    """
    Pytorch implementation of attention.
    Args:
        nn (nn.Module): module.
    """

    def __init__(
        self, dim: int, heads: int = 8, dim_head: int = 16, dropout: float = 0.0
    ):
        """
        Attention.
        Args:
            dim (int): Number of dimensions.
            heads (int, optional): Number of attention heads. Defaults to 8.
            dim_head (int, optional): Dimension of attention heads. Defaults to 16.
            dropout (float, optional): Degree of dropout. Defaults to 0.0.
        """
        super().__init__()
        inner_dim = dim_head * heads
        self.heads = heads
        self.scale = dim_head**-0.5

        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias=False)
        self.to_out = nn.Linear(inner_dim, dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass of attention module.
        Args:
            x (torch.Tensor): input tensor.
        Returns:
            torch.Tensor: output tensor.
        """
        h = self.heads
        q, k, v = self.to_qkv(x).chunk(3, dim=-1)
        q, k, v = map(lambda t: rearrange(t, "b n (h d) -> b h n d", h=h), (q, k, v))
        sim = einsum("b h i d, b h j d -> b h i j", q, k) * self.scale

        attn = sim.softmax(dim=-1)
        attn = self.dropout(attn)

        out = einsum("b h i j, b h j d -> b h i d", attn, v)
        out = rearrange(out, "b h n d -> b n (h d)", h=h)
        return self.to_out(out)


class Transformer(nn.Module):
    """
    Transformer.
    Based on paper:
    https://arxiv.org/abs/1706.03762
    Args:
        nn (nn.Module): Module with transformer.
    """

    def __init__(
        self,
        num_tokens: int,
        dim: int,
        depth: int,
        heads: int,
        dim_head: int,
        attn_dropout: float,
        ff_dropout: float,
    ):
        """
        Classical transformer.
        Args:
            num_tokens (int): Number of tokens i. e., unique classes + special tokens.
            dim (int): Number of dimensions.
            depth (int): Depth of encoder / decoder.
            heads (int): Number of attention heads.
            dim_head (int): Dimensions of attention heads.
            attn_dropout (float): Degree of dropout in attention.
            ff_dropout (float): Degree of dropout in feed-forward network.
        """
        super().__init__()
        self.embeds = nn.Embedding(num_tokens, dim)  # (Embed the categorical features.)
        self.layers = nn.ModuleList([])

        for _ in range(depth):
            self.layers.append(
                nn.ModuleList(
                    [
                        Residual(
                            PreNorm(
                                dim,
                                Attention(
                                    dim,
                                    heads=heads,
                                    dim_head=dim_head,
                                    dropout=attn_dropout,
                                ),
                            )
                        ),
                        Residual(PreNorm(dim, FeedForward(dim, dropout=ff_dropout))),
                    ]
                )
            )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass of transformer.
        Args:
            x (torch.Tensor): input tensor.
        Returns:
            torch.Tensor: output tensor.
        """
        x = self.embeds(x)

        for attn, ff in self.layers:  # type: ignore
            x = attn(x)
            x = ff(x)

        return x


class MLP(nn.Module):
    """
    Pytorch model of a vanilla multi-layer perceptron.
    Args:
        nn (nn.Module): module with implementation of MLP.
    """

    def __init__(self, dims: list[int], act: str | Callable[..., nn.Module]):
        """
        Multilayer perceptron.
        Depth of network is given by `len(dims)`. Capacity is given by entries
        of `dim`. Activation function is used after each linear layer. There is
        no activation function for the final linear layer, as it is sometimes part
        of the loss function already e. g., `nn.BCEWithLogitsLoss()`.
        Args:
            dims (List[int]): List with dimensions of layers.
            act (str | Callable[..., nn.Module]): Activation function of each linear
            layer.
        """
        super().__init__()
        dims_pairs = list(zip(dims[:-1], dims[1:]))
        layers = []
        for dim_in, dim_out in dims_pairs:
            linear = nn.Linear(dim_in, dim_out)
            layers.append(linear)
            layers.append(act)

        # drop last layer, as a sigmoid layer is included from BCELogitLoss
        del layers[-1]

        self.mlp = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward propagate tensor through MLP.
        Args:
            x (torch.Tensor): input tensor.
        Returns:
            torch.Tensor: output tensor.
        """
        return self.mlp(x)


class TabTransformer(nn.Module):
    """
    PyTorch model of TabTransformer.
    Based on paper:
    https://arxiv.org/abs/2012.06678
    Args:
        nn (nn.Module): Module with implementation of TabTransformer.
    """

    def __init__(
        self,
        *,
        categories: tuple[int, ...] | tuple[()],
        num_continuous: int,
        dim: int = 32,
        depth: int = 4,
        heads: int = 8,
        dim_head: int = 16,
        dim_out: int = 1,
        mlp_hidden_mults: tuple[(int, int)] = (4, 2),
        mlp_act: str | Callable[..., nn.Module] = nn.ReLU,
        num_special_tokens: int = 2,
        continuous_mean_std: torch.Tensor | None = None,
        attn_dropout: float = 0.0,
        ff_dropout: float = 0.0,
    ):
        """
        TabTransformer.
        Originally introduced in https://arxiv.org/abs/2012.06678.
        Args:
            categories ([List[int] | Tuple[()]): List with number of categories
            for each categorical feature. If no categorical variables are present,
            use empty tuple. For categorical variables e. g., option type ('C' or 'P'),
            the list would be `[1]`.
            num_continuous (int): Number of continous features.
            dim (int, optional): Dimensionality of transformer. Defaults to 32.
            depth (int, optional): Depth of encoder / decoder of transformer.
            Defaults to 4.
            heads (int, optional): Number of attention heads. Defaults to 8.
            dim_head (int, optional): Dimensionality of attention head. Defaults to 16.
            dim_out (int, optional): Dimension of output layer of MLP. Set to one for
            binary classification. Defaults to 1.
            mlp_hidden_mults (Tuple[(int, int)], optional): multipliers for dimensions
            of hidden layer in MLP. Defaults to (4, 2).
            mlp_act (str | Callable[..., nn.Module], optional): Activation function used
            in MLP. Defaults to nn.ReLU().
            num_special_tokens (int, optional): Number of special tokens in transformer.
            Defaults to 2.
            continuous_mean_std (Optional[torch.Tensor]): List with mean and
            std deviation of each continous feature. Shape eq. `[num_continous x 2]`.
            Defaults to None.
            attn_dropout (float, optional): Degree of attention dropout used in
            transformer. Defaults to 0.0.
            ff_dropout (float, optional): Dropout in feed forward net. Defaults to 0.0.
        """
        super().__init__()
        assert all(
            map(lambda n: n > 0, categories)
        ), "number of each category must be positive"

        # categories related calculations

        self.num_categories = len(categories)
        self.num_unique_categories = sum(categories)

        # create category embeddings table

        self.num_special_tokens = num_special_tokens
        total_tokens = self.num_unique_categories + num_special_tokens

        # for automatically offsetting unique category ids to the correct position
        #  in the categories embedding table

        categories_offset = F.pad(
            torch.tensor(list(categories)), (1, 0), value=num_special_tokens
        )  # Prepend num_special_tokens.
        categories_offset = categories_offset.cumsum(dim=-1)[:-1]
        self.register_buffer("categories_offset", categories_offset)

        # continuous

        if continuous_mean_std is not None:
            assert continuous_mean_std.shape == (num_continuous, 2,), (
                f"continuous_mean_std must have a shape of ({num_continuous}, 2)"
                f"where the last dimension contains the mean and variance respectively"
            )
        self.register_buffer("continuous_mean_std", continuous_mean_std)

        self.norm = nn.LayerNorm(num_continuous)
        self.num_continuous = num_continuous

        # transformer

        self.transformer = Transformer(
            num_tokens=total_tokens,
            dim=dim,
            depth=depth,
            heads=heads,
            dim_head=dim_head,
            attn_dropout=attn_dropout,
            ff_dropout=ff_dropout,
        )

        # mlp to logits

        input_size = (dim * self.num_categories) + num_continuous
        j = input_size // 8

        hidden_dimensions = list(map(lambda t: j * t, mlp_hidden_mults))
        all_dimensions = [input_size, *hidden_dimensions, dim_out]

        self.mlp = MLP(all_dimensions, act=mlp_act)

    def forward(self, x_categ: torch.Tensor, x_cont: torch.Tensor) -> torch.Tensor:
        """
        Forward pass of TabTransformer.
        Args:
            x_categ (torch.Tensor): tensor with categorical data.
            x_cont (torch.Tensor): tensor with continous data.
        Returns:
            torch.Tensor: predictions with shape [batch, 1]
        """
        # Adaptation to work without categorical data
        if x_categ is not None:
            assert x_categ.shape[-1] == self.num_categories, (
                f"you must pass in {self.num_categories} "
                f"values for your categories input"
            )
            x_categ += self.categories_offset
            x = self.transformer(x_categ)
            flat_categ = x.flatten(1)

        assert x_cont.shape[1] == self.num_continuous, (
            f"you must pass in {self.num_continuous} "
            f"values for your continuous input"
        )

        if self.continuous_mean_std is not None:
            mean, std = self.continuous_mean_std.unbind(dim=-1)  # type: ignore
            x_cont = (x_cont - mean) / std

        normed_cont = self.norm(x_cont)

        # Adaptation to work without categorical data
        x = (
            torch.cat((flat_categ, normed_cont), dim=-1)
            if x_categ is not None
            else normed_cont
        )

        return self.mlp(x)

In [9]:
"""
Implementation of a dataset for tabular data.
Supports both categorical and continous data.
"""

from __future__ import annotations

import pandas as pd
import torch
from torch.utils.data import Dataset


class TabDataset(Dataset):
    """PyTorch Dataset for tabular data.
    Args:
        Dataset (Dataset): dataset
    """

    def __init__(
        self,
        x: pd.DataFrame,
        y: pd.Series,
        cat_features: list[str] | None = None,
        cat_unique_counts: tuple[int, ...] | None = None,
    ):
        """
        Tabular data set holding data for the model.
        Args:
            x (pd.DataFrame): feature matrix.
            y (pd.Series): target.
            cat_features (Optional[List[str]], optional): List with categorical columns.
            Defaults to None.
            cat_unique_counts (Optional[Tuple[int]], optional): Number of categories per
            categorical feature. Defaults to None.
        """
        self._cat_unique_counts = () if not cat_unique_counts else cat_unique_counts

        # calculate cat indices
        features = x.columns.tolist()
        cat_features = [] if not cat_features else cat_features
        self._cat_idx = [features.index(i) for i in cat_features if i in features]

        # calculate cont indices
        cont_features = [f for f in features if f not in cat_features]
        self._cont_idx = [features.index(i) for i in cont_features if i in features]

        assert (
            x.shape[0] == y.shape[0]
        ), "Length of feature matrix must match length of target."
        assert len(cat_features) == len(
            self._cat_unique_counts
        ), "For all categorical features the number of unique entries must be provided."

        # adjust target to be either 0 or 1
        self.y = torch.tensor(y.values).float()
        self.y[self.y < 0] = 0

        # cut into continous and categorical tensor
        self.x_cat: torch.Tensor | None = None
        if len(self._cat_idx) > 0:
            self.x_cat = torch.tensor(x.iloc[:, self._cat_idx].values).int()
        self.x_cont = torch.tensor(x.iloc[:, self._cont_idx].values).float()

    def __len__(self) -> int:
        """
        Length of dataset.
        Returns:
            int: length
        """
        return len(self.x_cont)

    def __getitem__(
        self, idx: int
    ) -> tuple[torch.Tensor | None, torch.Tensor, torch.Tensor]:
        """
        Get sample for model.
        Args:
            idx (int): _description_
        Returns:
            Tuple[torch.Tensor | None, torch.Tensor, torch.Tensor]:
            x_cat (if present if present otherwise None), x_cont and y.
        """
        return (
            self.x_cat[idx] if self.x_cat else None,
            self.x_cont[idx],
            self.y[idx],
        )

In [10]:
"""
A fast dataloader-like object to load batches of tabular data sets.
Adapted from here:
https://discuss.pytorch.org/t/dataloader-much-slower-than-manual-batching/27014/6
"""
from __future__ import annotations

from typing import Any

import torch


class TabDataLoader:
    """
    PyTorch Implementation of a dataloader for tabular data.
    Due to a chunk-wise reading or several rows at once it is preferred
    over the standard dataloader that reads row-wise.
    """

    def __init__(
        self,
        *tensors: torch.Tensor | None,
        batch_size: int = 4096,
        shuffle: bool = False,
        device: str = "cpu",
        **kwargs: Any,
    ):
        """
        TabDataLoader.
        Tensors can be None e. g., if there is no categorical data.
        Args:
            batch_size (int, optional): size of batch. Defaults to 4096.
            shuffle (bool, optional): shuffle data. Defaults to False.
            device (str, optional): device where. Defaults to "cpu".
        """
        self.device = device
        # check for tensors that are None
        self.none_mask = tuple(t is None for t in tensors)
        # filter if for not none tensors
        self.tensors = tuple(t for t in tensors if t is not None)

        # check if all tensors have same length
        assert all(t.shape[0] == self.tensors[0].shape[0] for t in self.tensors)

        self.dataset_len = self.tensors[0].shape[0]
        self.batch_size = batch_size
        self.shuffle = shuffle

        # Calculate # batches
        n_batches, remainder = divmod(self.dataset_len, self.batch_size)
        if remainder > 0:
            n_batches += 1
        self.n_batches = n_batches

    def __iter__(self) -> TabDataLoader:
        """
        Return itself.
        Returns:
            TabDataLoader: TabDataLoader
        """
        if self.shuffle:
            r = torch.randperm(self.dataset_len)
            self.tensors = tuple(t[r] for t in self.tensors if t)
        # reset counter on new iteration
        self.i = 0
        return self

    def __next__(self) -> tuple[torch.Tensor | None, ...]:
        """
        Generate next batch with size of 'batch_size'.
        Batches can be underful.
        Raises:
            StopIteration: stopping criterion.
        Returns:
            Tuple[torch.Tensor | None, torch.Tensor, torch.Tensor]: (X_cat), X_cont, y
        """
        if self.i >= self.dataset_len:
            raise StopIteration
        mixed_batch: list[torch.Tensor | None] = [
            t[self.i : self.i + self.batch_size].to(self.device) for t in self.tensors
        ]
        self.i += self.batch_size

        # tensors + nones if input tensors contained none
        for i, is_none in enumerate(self.none_mask):
            if is_none:
                mixed_batch.insert(i, None)

        return tuple(mixed_batch)

    def __len__(self) -> int:
        """
        Get number of full and partial batches in data set.
        Returns:
            int: number of batches.
        """
        return self.n_batches

In [13]:
# https://wandb.ai/fbv/thesis/runs/4fmccjm7/files/wandb-summary.json
# https://wandb.ai/fbv/thesis/artifacts/model/3lfsbuby_TabTransformer_default_trial_82.pth/3a1937a3e6ec748d45a3/metadata
params = {  "dim": 32,
  "depth": 3,
  "heads": 2,
  "weight_decay": 0.00835620489462654,
  "lr": 0.0015514372468568292,
  "dropout": 0.1,
  "batch_size": 32768}

training_data = TabDataset(X_test, y_test, [], [])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")        

# differentiate between continous features only and mixed.
test_loader = TabDataLoader(
training_data.x_cat, training_data.x_cont, training_data.y, batch_size=params['batch_size'], device=device
)

       
model = TabTransformer(
            categories=[],
            num_continuous=len(features_classical_size),
            dim_out=1,
            mlp_act=nn.ReLU(),
            dim=params["dim"],
            depth=params["depth"],
            heads=params["heads"],
            attn_dropout=params["dropout"],
            ff_dropout=params["dropout"],
            mlp_hidden_mults=(4, 2),
        ).to(device)


model.load_state_dict(torch.load(Path(model_dir,"3lfsbuby_TabTransformer_default_trial_82.pth")))
model.eval()

y_pred, y_true = [], []

for x_cat, x_cont, targets in test_loader:
  output = model(x_cat, x_cont)

  # map between zero and one, sigmoid is otherwise included in loss already
  # https://stackoverflow.com/a/66910866/5755604
  output = torch.sigmoid(output.squeeze())
  y_pred.append(output.detach().cpu().numpy())
  y_true.append(targets.detach().cpu().numpy())  # type: ignore

# round prediction to nearest int
y_pred = np.rint(np.concatenate(y_pred))
y_true = np.concatenate(y_true)

# map zeros back to -1
y_pred[y_pred == 0] = -1
y_true[y_true == 0] = -1

In [14]:
accuracy_score(y_true, y_pred)

0.672905426069829

In [15]:
# load default data to use unscaled version with all possible columns
test_orig = pd.read_parquet(
    f"gs://thesis-bucket-option-trade-classification/data/preprocessed/test_set_extended_20.parquet",
    engine="fastparquet",
)

In [16]:
# Copy unscaled columns
X_print = test_orig.copy()

# add baseline results
X_print["rule"] = "Baseline"
X_print["buy_sell_predicted"] = y_pred

# prepare columns for printing
X_print["ttm"] = (
    X_print["EXPIRATION"].dt.to_period("M")
    - X_print["QUOTE_DATETIME"].dt.to_period("M")
).apply(lambda x: x.n)
X_print["year"] = X_print["QUOTE_DATETIME"].dt.year

bins_tradesize = [-np.inf, 1, 3, 5, 11, np.inf]
trade_size_labels = ["(0,1]", "(1,3]", "(3,5]", "(5,11]", ">11"]
X_print["TRADE_SIZE_binned"] = pd.cut(
    X_print["TRADE_SIZE"], bins_tradesize, labels=trade_size_labels
)

bins_years = [2004, 2007, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
year_labels = [
    "2005-2007",
    "2008-2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
]
X_print["year_binned"] = pd.cut(X_print["year"], bins_years, labels=year_labels)

bins_ttm = [-np.inf, 1, 2, 3, 6, 12, np.inf]
ttm_labels = [
    "ttm <= 1 month",
    "ttm (1-2] month",
    "ttm (2-3] month",
    "ttm (3-6] month",
    "ttm (6-12] month",
    "ttm > 12 month",
]
X_print["ttm_binned"] = pd.cut(X_print["ttm"], bins_ttm, labels=ttm_labels)


bins_myn = [-np.inf, 0.7, 0.9, 1.1, 1.3, np.inf]
myn_labels = [
    "mny <=0.7",
    "mny (0.7-0.9]",
    "mny (0.9-1.1]",
    "mny (1.1-1.3]",
    "mny > 1.3",
]
X_print["myn_binned"] = pd.cut(X_print["myn"], bins_myn, labels=myn_labels)

X_print["issue_type_binned"] = X_print["issue_type"].replace(
    {"0": 'Stock options', 'A': 'Index options', '7': 'Others',
     'F': 'Others', '%': 'Others', ' ': 'Others'})


# TODO: time from previous trade; same underlying or any?


In [17]:
def check_robustness(criterion: str = "year_binned") -> pd.DataFrame:
    """
    Check robustness of rules by calculating the accuracy for a given
    criterion and rules.

    Example:
    rule		Baseline
    TRADE_SIZE_binned
    (0,1]	  0.710966
    (1,3]	  0.717664
    (3,5]	  0.715195
    (5,11]	0.699428
    >11	  	0.688348

    Args:
        criterion (str, optional): criterion to check robustness for.
        Defaults to "year_binned".

    Returns:
        pd.DataFrame: DataFrame with accuracy of rules. Rule in columns and
        criterion values in rows.
    """

    # fill others randomly with equal weight for every class.
    X_print["buy_sell_predicted"] = X_print["buy_sell_predicted"].map(
        lambda l: l if not np.isnan(l) else np.random.choice([-1, 1])
    )

    # cuculate average over columns if multiple subsets are combined
    results = (
        X_print.groupby(["rule", criterion])[["buy_sell", "buy_sell_predicted"]]
        .apply(lambda x: accuracy_score(x["buy_sell"], x["buy_sell_predicted"]))
        .unstack(level=0)
        .assign(avg=lambda x: x.mean(axis=1))
    )
    return results


In [18]:
check_robustness("year_binned")


rule,Baseline,avg
year_binned,,
2015,0.645190,0.645190
2016,0.676413,0.676413
2017,0.674097,0.674097


In [19]:
check_robustness("OPTION_TYPE")


rule,Baseline,avg
OPTION_TYPE,,
C,0.675753,0.675753
P,0.669654,0.669654


In [20]:
check_robustness("issue_type_binned")

rule,Baseline,avg
issue_type_binned,,
Index options,0.524634,0.524634
Others,0.702792,0.702792
Stock options,0.663093,0.663093


In [21]:
check_robustness("TRADE_SIZE_binned")


rule,Baseline,avg
TRADE_SIZE_binned,,
"(0,1]",0.683563,0.683563
"(1,3]",0.690546,0.690546
"(3,5]",0.688925,0.688925
"(5,11]",0.653614,0.653614
>11,0.634339,0.634339


In [22]:
check_robustness("ttm_binned")


rule,Baseline,avg
ttm_binned,,
ttm <= 1 month,0.669447,0.669447
ttm (1-2] month,0.691056,0.691056
ttm (2-3] month,0.681317,0.681317
ttm (3-6] month,0.675536,0.675536
ttm (6-12] month,0.675719,0.675719
ttm > 12 month,0.665156,0.665156


In [23]:
check_robustness("myn_binned")


rule,Baseline,avg
myn_binned,,
mny <=0.7,0.665539,0.665539
mny (0.7-0.9],0.665949,0.665949
mny (0.9-1.1],0.683752,0.683752
mny (1.1-1.3],0.626057,0.626057
mny > 1.3,0.603918,0.603918
